# Solving the Diet Problem for Armenia's Five Markets

In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import linprog

In [2]:
NUTRITION_FILE = '../data/nutrition.csv'
PRICING_FILE = '../data/pricing.csv'

# If a particular market does not have a food,
# we set its price to be "infinity" (i.e. 999999).
INFINITY = 999999

In [3]:
nutrition = pd.read_csv(NUTRITION_FILE, index_col=0)
pricing = pd.read_csv(PRICING_FILE, index_col=0)

In [4]:
grouped = (pricing[pricing.country_name == 'Armenia']
                  .groupby(['country_name', 'locality_name', 'market_name']))

In [5]:
solns = {}

for market, idx in grouped.groups.items():
    df = pricing.loc[idx]

    A_ub = -np.transpose(nutrition.values)
    b_ub = -np.array([65, 300, 25, 50])

    # Construct c appropriately (i.e. add 0s to the appropriate foods)
    c = pd.Series(data=INFINITY*np.ones(84), index=nutrition.index)
    c.loc[df.commodity_name] = df.price.values
    c = c.values

    solns[market] = linprog(c, A_ub, b_ub)

In [6]:
# Collect optimization status, minimum value and minimum into one array
data = np.hstack([
    np.transpose(
        np.vstack([[soln.status for soln in solns.values()],
                   [soln.fun for soln in solns.values()]])
    ),
    [soln.x for soln in solns.values()]
])

In [7]:
df = pd.DataFrame(data=data,
                  index=solns.keys(),
                  columns=['status', 'fun'] + nutrition.index.tolist())

In [8]:
df.to_csv('armenia.csv')

## Sensitivity Analysis

Here, we focus our attention on Armavir, Lori, Armenia.

In [9]:
x = df.iloc[0]

In [10]:
# Only eggs and pasta...
x[x != 0]

fun      277.556244
Eggs       0.660464
Pasta      0.561798
Name: (Armenia, Lori, Armavir), dtype: float64

In [11]:
mask = (pricing.country_name == 'Armenia') & (pricing.locality_name == 'Lori') & (pricing.market_name == 'Armavir')
pricing = pricing[mask].drop(['country_name', 'locality_name', 'market_name'], axis=1).reset_index(drop=True)

In [12]:
nutrition

,Protein,Fat,Carb,Fiber
Bread,106.7,45.3,475.4,40.0
Rice,39.9,3.4,213.4,18.0
Wheat,74.9,12.7,425.3,11.0
Livestock,174.4,81.0,0.4,0.0
Apples,9.3,3.2,658.9,87.0
Bananas,38.9,18.1,882.8,99.0
Beans,18.0,32.0,28.0,1.0
Carrots,9.3,2.4,95.8,28.0
Cheese,269.6,309.9,14.4,0.0
Eggs,480.5,439.0,11.3,0.0


In [13]:
pricing

,commodity_name,currency_name,market_type,unit_of_goods_measurement,price
0,Apples,AMD,Retail,KG,650.0
1,Bread,AMD,Retail,KG,400.0
2,Cabbage,AMD,Retail,KG,310.0
3,Carrots,AMD,Retail,KG,375.0
4,Cheese,AMD,Retail,KG,2000.0
5,Cucumbers,AMD,Retail,KG,615.0
6,Eggs,AMD,Retail,Unit,80.0
7,Fish,AMD,Retail,KG,1850.0
8,Lentils,AMD,Retail,KG,515.0
9,Meat,AMD,Retail,KG,2650.0


In [15]:
A_ub = -np.transpose(nutrition.values)
b_ub = -np.array([65, 300, 25, 50])

# Construct c appropriately (i.e. add 0s to the appropriate foods)
c = pd.Series(data=INFINITY*np.ones(84), index=nutrition.index)
c.loc[pricing.commodity_name] = pricing.price.values
c = c.values

linprog(c, A_ub, b_ub)

     fun: 277.55624376435344
 message: 'Optimization terminated successfully.'
     nit: 15
   slack: array([332.63398684,   0.        , 393.75538123,   0.        ])
  status: 0
 success: True
       x: array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.66046428,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.56179775, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.   